In [48]:
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle
import tensorflow
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re
from nltk.stem.porter import PorterStemmer

In [65]:
model = keras.models.load_model('saved_model')
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [67]:
Y_columns = ['Contract', 'Criminal', 'Family', 'Property', 'Tort_1', 'Tort_2']
Y_columns = sorted(Y_columns)

In [66]:
ps = PorterStemmer()
sequence_len = 20

def model_predict(text):
    print('@ Sentence input:', text)

    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    text = text.split()
    text = [ps.stem(word) for word in text]
    text = ' '.join(text)
    
    sequence = tokenizer.texts_to_sequences( [text] )[0]

    if (len(sequence) == 0):
        return Y_columns

    if len(sequence) < sequence_len:
        # sequence = [0] * (sequence_len - len(sequence)) + sequence
        num_repeats = (sequence_len // len(sequence)) + 1
        sequence = sequence * num_repeats
#     print('@ sequence:', sequence)

    sequences = []
    num_sequences = len(sequence) // sequence_len
    
    for i in range(num_sequences):
        start = sequence_len * i
        end = sequence_len * (i + 1)
        sequences.append(sequence[start:end])
    
    y_prob = model.predict(sequences)
    y_avg_prob = list(np.average(y_prob, axis=0))

    y_avg_prob_map = { Y_columns[i] : y_avg_prob[i] for i in range(len(y_avg_prob)) }
#     print('Map:', y_avg_prob_map)

    y_avg_prob_map_sorted = {k: v for k, v in sorted(y_avg_prob_map.items(), key=lambda x: x[1], reverse=True)}
    print('Sorted Map:', y_avg_prob_map_sorted)
    
    category_list = list(y_avg_prob_map_sorted.keys())[0:2]
#     print('Category list: ', category_list)

    return category_list

In [76]:
test_sentences = [
#     'Divorcing my wife',
#     'Abused by my husband',
#     'I want to create a will',
#     'Mental health dropping',
#     'I wish to adopt a child',
    
#     'I have an employment issue',
#     'Want to Claim insurance',
#     'I took out a large loan',
#     'I need some help on an investment I made',
#     'I bought a product and need to use lemon law',
    
#     'I intend to buy a BTO house',
#     'paying my mortgage',
#     'I had a huge fight with my tenant. I am not sure what I should do. He keeps screaming at me and does not reply me',
#     'I got sued for a copyright',
#     'I had a will estate planning issue',
    
#     'I was harassed by some stranger on the street',
#     'I received a defamation letter',
#     'I have issues with privacy',
#     'PDPA is a serious concern my company faces',
#     'My neighbor keeps banging my door',
    
    'I wish to make a personal injury claim',
    'I was threatened by the man earlier',
    'My employer neglected my well-being',
    'I got into a traffic accident',
    'My doctor did not prescribe the right medicine',
]

for sentence in test_sentences:
    print( model_predict(sentence) )

@ Sentence input: I wish to make a personal injury claim
Sorted Map: {'Tort_2': 0.9994779, 'Contract': 0.0003865548, 'Property': 7.4390715e-05, 'Criminal': 4.254056e-05, 'Family': 1.781766e-05, 'Tort_1': 7.997987e-07}
['Tort_2', 'Contract']
@ Sentence input: I was threatened by the man earlier
Sorted Map: {'Tort_2': 0.9993742, 'Criminal': 0.0003311706, 'Contract': 0.00013341244, 'Property': 0.00012338246, 'Tort_1': 3.1148287e-05, 'Family': 6.661747e-06}
['Tort_2', 'Criminal']
@ Sentence input: My employer neglected my well-being
Sorted Map: {'Contract': 0.9886607, 'Family': 0.01096761, 'Criminal': 0.00032290703, 'Tort_2': 3.315172e-05, 'Property': 1.5195589e-05, 'Tort_1': 4.8031194e-07}
['Contract', 'Family']
@ Sentence input: I got into a traffic accident
Sorted Map: {'Tort_2': 0.99970883, 'Criminal': 0.0001083983, 'Contract': 9.287373e-05, 'Property': 6.9599555e-05, 'Tort_1': 1.9526706e-05, 'Family': 7.433789e-07}
['Tort_2', 'Criminal']
@ Sentence input: My doctor did not prescribe t